# Data Science Capstone Project

## INDIAN RESTAURANT IN CANADA

### 1.Collect the data and tranform it into dataframe
### 2.Get the location of neighbourhood
### 3.Get the venues of the indian restaurant
### 4.Divide the neighbouhood into different clusters
### 5.Select the best cluster depending on availabilty of restaurant in that cluster

## Import all libraries 

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files
!conda install -c conda-forge geocoder --yes
import geocoder

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print("Libraries imported.")

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


## Get the data and covert into dataframe

In [48]:
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
toronto=pd.DataFrame(df[0])
toronto.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


## Eliminating the unnecessary columns

In [52]:
toronto_df=toronto[toronto.Borough != "Not assigned"].reset_index(drop=True)
toronto_df.drop(['Postal code','Borough'],1,inplace=True)
toronto_df.head()

,Neighborhood
0,Parkwoods
1,Victoria Village
2,Regent Park / Harbourfront
3,Lawrence Manor / Lawrence Heights
4,Queen's Park / Ontario Provincial Government


## Getting the location of neighbourhood

In [53]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, toronto , canada'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

latlong = [ get_latlng(neighborhood) for neighborhood in toronto_df["Neighborhood"].tolist() ]
latlong

[[43.68657452246789, -79.40999250225033],
 [43.73154000000005, -79.31427999999994],
 [43.65974052181429, -79.3615640533021],
 [43.72357000000005, -79.43710999999996],
 [43.666630269551845, -79.39326783273027],
 [43.66321327878551, -79.53150965640926],
 [43.81023000000005, -79.22037999999998],
 [43.705680011617886, -79.33385830530915],
 [43.70626000000004, -79.30090999999999],
 [43.65794000000005, -79.37561999999997],
 [43.70842942383775, -79.44648154184813],
 [43.65297000000004, -79.55741999999998],
 [43.789480000000026, -79.17613999999998],
 [43.705680011617886, -79.33385830530915],
 [43.68752000000006, -79.32058999999998],
 [43.67110000000008, -79.37358999999998],
 [43.68942000000004, -79.42697999999996],
 [43.63349000000005, -79.57073999999994],
 [43.76602147574681, -79.18538783940997],
 [43.67413000000005, -79.29643999999996],
 [43.64829993392594, -79.37524996335624],
 [43.68857000000003, -79.45482999999996],
 [43.767480000000035, -79.22828999999996],
 [43.70023937855272, -79.35106

## Converting the location of neighbourhood into dataframe

In [54]:
toronto_df_coord = pd.DataFrame(latlong, columns=['Latitude', 'Longitude'])
toronto_df['Latitude']=toronto_df_coord['Latitude']
toronto_df['Longitude']=toronto_df_coord['Longitude']

print(toronto_df.shape)
toronto_df.head()

(103, 3)


,Neighborhood,Latitude,Longitude
0,Parkwoods,43.686575,-79.409993
1,Victoria Village,43.731540,-79.314280
2,Regent Park / Harbourfront,43.659741,-79.361564
3,Lawrence Manor / Lawrence Heights,43.723570,-79.437110
4,Queen's Park / Ontario Provincial Government,43.666630,-79.393268


## Getting the address of Toronto

In [55]:

address = 'toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of toronto are 43.6534817, -79.3839347.


## Map of Toronto with neighbourhood superimposed on top

In [56]:

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# adding markers to map
for lat, lng, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

## Using the Foursquare API to explore the neighborhoods in selected borough in Toronto

In [57]:
CLIENT_ID = 'SSUTEMIRZRUDPXKK4GGPBMLA1PLYN5TX2UZG3XS4ODTHAB4S' # your Foursquare ID
CLIENT_SECRET = 'UJD4CGNIHVB50WXNJYO2S3W3BTXORDR0ADTEIDBFBFHABXNW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SSUTEMIRZRUDPXKK4GGPBMLA1PLYN5TX2UZG3XS4ODTHAB4S
CLIENT_SECRET:UJD4CGNIHVB50WXNJYO2S3W3BTXORDR0ADTEIDBFBFHABXNW


## Getting the top 300 venues that are within a radius of 5000 meters of Toronto

In [58]:
radius = 5000
LIMIT = 300

venues = []

for lat, long, neighborhood in zip( toronto_df['Latitude'],  toronto_df['Longitude'], toronto_df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

## Converting the venues into dataframe

In [59]:
# converting the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(10205, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Parkwoods,43.686575,-79.409993,Booyah,43.681744,-79.418011,Ice Cream Shop
1,Parkwoods,43.686575,-79.409993,Scaramouche,43.681293,-79.399492,French Restaurant
2,Parkwoods,43.686575,-79.409993,LCBO,43.681497,-79.391261,Liquor Store
3,Parkwoods,43.686575,-79.409993,Aroma Espresso Bar,43.688170,-79.412599,Café
4,Parkwoods,43.686575,-79.409993,Cedarvale Park,43.692535,-79.428705,Field


In [60]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Agincourt,100,100,100,100,100,100
Alderwood / Long Branch,100,100,100,100,100,100
Bathurst Manor / Wilson Heights / Downsview North,100,100,100,100,100,100
Bayview Village,100,100,100,100,100,100
Bedford Park / Lawrence Manor East,100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
Birch Cliff / Cliffside West,100,100,100,100,100,100
Brockton / Parkdale Village / Exhibition Place,100,100,100,100,100,100
Business reply mail Processing CentrE,100,100,100,100,100,100


In [61]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 240 uniques categories.


## Displaying the unique venues

In [64]:
venues_df['VenueCategory'].unique()

array(['Ice Cream Shop', 'French Restaurant', 'Liquor Store', 'Café',
       'Field', 'Coffee Shop', 'Indian Restaurant', 'Juice Bar', 'Park',
       'Bagel Shop', 'Spa', 'Gastropub', 'BBQ Joint',
       'Italian Restaurant', 'Sushi Restaurant', 'Trail', 'Grocery Store',
       'Vegetarian / Vegan Restaurant', 'Concert Hall',
       'Tapas Restaurant', 'Indonesian Restaurant', 'Tea Room', 'Hotel',
       'Athletics & Sports', 'General Entertainment', 'Restaurant',
       'Gourmet Shop', 'Shopping Mall', 'Beer Bar', 'Health Food Store',
       'Bakery', 'Brewery', 'Dessert Shop', 'Bar', 'Dance Studio',
       'Yoga Studio', 'Supermarket', 'South American Restaurant',
       'Middle Eastern Restaurant', 'Cupcake Shop', 'Wine Bar', 'Hostel',
       'Deli / Bodega', 'Plaza', 'Bubble Tea Shop', 'Gym', 'Music Store',
       'Historic Site', 'Farmers Market', 'Arts & Crafts Store',
       'Mexican Restaurant', 'Organic Grocery', 'Art Gallery',
       'Cocktail Bar', 'Flower Shop', 'History Mu

## Analyzing the neighbourhgood and venues found in that neighbourhood

In [65]:
# one hot encoding
tnt_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe

tnt_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhoods column to the first column
fixed_columns = list(tnt_onehot.columns[-1:]) + list(tnt_onehot.columns[:-1])
tnt_onehot = tnt_onehot[fixed_columns]

print(tnt_onehot.shape)
tnt_onehot.head()

(10205, 241)


,Neighborhoods,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Campground,Cantonese Restaurant,Caribbean Restaurant,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hostel,Hotel,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Rock Climbing Spot,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Ski Chalet,Snack Place,Soccer Field,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

## Grouping data and taking their mean

In [66]:

tnt_grouped = tnt_onehot.groupby(['Neighborhoods']).mean().reset_index()
print(tnt_grouped.shape)
tnt_grouped.head()

(98, 241)


,Neighborhoods,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Campground,Cantonese Restaurant,Caribbean Restaurant,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hostel,Hotel,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Rock Climbing Spot,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Ski Chalet,Snack Place,Soccer Field,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Agincourt,0.0,0.00,0.0,0.01,0.0,0.0,0.0,0.01,0.02,0.00,0.0,0.00,0.01,0.0,0.00,0.05,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.01,0.0,0.0,0.0,0.02,0.00,0.00,0.0,0.01,0.04,0.02,0.00,0.00,0.0,0.02,0.07,0.0,0.0,0.0,0.0,0.07,0.00,0.0,0.00,0.00,0.00,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.01,0.0,0.01,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.02,0.0,0.0,0.02,0.0,0.00,0.01,0.01,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.01,0.00,0.0,0.00,0.00,0.00,0.0,0.00,

## Finding the neighbourhood that has INDIAN RESTAURANT

In [71]:
print(len(tnt_grouped[tnt_grouped["Indian Restaurant"] > 0]))
tnt_ind_res = tnt_grouped[["Neighborhoods","Indian Restaurant"]]
tnt_ind_res.head()

58


,Neighborhoods,Indian Restaurant
0,Agincourt,0.02
1,Alderwood / Long Branch,0.02
2,Bathurst Manor / Wilson Heights / Downsview North,0.00
3,Bayview Village,0.00
4,Bedford Park / Lawrence Manor East,0.06


In [73]:
temp=[]
for neighborhood,VenueName, category in zip(venues_df['Neighborhood'],venues_df['VenueName'],venues_df['VenueCategory']):
    if(category==("Indian Restaurant")):
        temp.append([neighborhood ,VenueName, category])
ind_res=pd.DataFrame(temp)
ind_res.rename(columns={0:"Neighborhood",1:"VenueName",2:"VenueCategory"},inplace=True)
print(ind_res.shape)
ind_res.head()

(142, 3)


,Neighborhood,VenueName,VenueCategory
0,Parkwoods,Pukka Restaurant,Indian Restaurant
1,Parkwoods,Marigold Indian Bistro,Indian Restaurant
2,Victoria Village,Sultan Of Samosas,Indian Restaurant
3,Lawrence Manor / Lawrence Heights,Pukka Restaurant,Indian Restaurant
4,Islington Avenue,Curry Twist,Indian Restaurant


## CLUSTERING

In [74]:

# set number of clusters
kclusters = 9
tnt_ind_res_clustering = tnt_ind_res.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tnt_ind_res_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:]

array([1, 1, 0, 0, 7, 0, 3, 0, 0, 0, 1, 0, 2, 0, 3, 0, 5, 5, 0, 1, 4, 1,
       4, 2, 0, 3, 3, 0, 3, 0, 4, 0, 1, 4, 8, 0, 4, 3, 5, 4, 3, 1, 3, 5,
       0, 0, 3, 1, 1, 3, 6, 5, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       3, 4, 1, 3, 3, 5, 0, 0, 0, 1, 0, 0, 1, 0, 1, 4, 1, 4, 0, 0, 0, 3,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 3], dtype=int32)

In [77]:

# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
tnt_merged = tnt_ind_res.copy()

# add clustering labels
tnt_merged["Cluster Labels"] = kmeans.labels_

#blr_merged = blr_merged.join(icecream.drop(["VenueCategory"], 1).set_index("Neighborhood"), on="Neighborhood")


print(tnt_merged.shape)
tnt_merged.head()

(98, 3)


,Neighborhoods,Indian Restaurant,Cluster Labels
0,Agincourt,0.02,1
1,Alderwood / Long Branch,0.02,1
2,Bathurst Manor / Wilson Heights / Downsview North,0.00,0
3,Bayview Village,0.00,0
4,Bedford Park / Lawrence Manor East,0.06,7


In [108]:

# sort the results by Cluster Labels
print(tnt_merged.shape)
tnt_merged.sort_values(["Cluster Labels"], inplace=True)
tnt_merged.rename(columns={"Neighborhoods":"Neighborhood"},inplace=True)

tnt_merged.head()

(119, 5)


,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
85,University of Toronto / Harbord,0.0,0,43.66320,-79.401290
76,Stn A PO Boxes,0.0,0,43.64869,-79.385440
31,"Garden District, Ryerson",0.0,0,43.65794,-79.375620
63,Queen's Park / Ontario Provincial Government,0.0,0,43.66663,-79.393268
2,Bathurst Manor / Wilson Heights / Downsview North,0.0,0,43.73737,-79.434170


In [109]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat,lon,poi, cluster in zip(tnt_merged['Latitude'],tnt_merged['Longitude'],tnt_merged['Neighborhood'],tnt_merged['Cluster Labels']):
    label = folium.Popup('{} - Cluster {}'.format(poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining the clusters

In [105]:
#cluster 1
print(tnt_merged.loc[tnt_merged['Cluster Labels'] == 0].shape)

tnt_merged.loc[tnt_merged['Cluster Labels'] == 0]

(58, 5)


,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
85,University of Toronto / Harbord,0.0,0,43.663200,-79.401290
58,Northwood Park / York University,0.0,0,43.742650,-79.496470
59,Old Mill South / King's Mill Park / Sunnylea /...,0.0,0,43.648690,-79.385440
24,Downsview,0.0,0,43.720188,-79.499920
24,Downsview,0.0,0,43.720188,-79.499920
24,Downsview,0.0,0,43.720188,-79.499920
24,Downsview,0.0,0,43.720188,-79.499920
24,Downsview,0.0,0,43.720188,-79.499920
35,Harbourfront East / Union Station / Toronto Is...,0.0,0,43.798896,-79.146162
24,Downsview,0.0,0,43.720188,-79.499920


In [110]:
#cluster 2
print(tnt_merged.loc[tnt_merged['Cluster Labels'] == 1].shape)

tnt_merged.loc[tnt_merged['Cluster Labels'] == 1]

(23, 5)


,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
1,Alderwood / Long Branch,0.02,1,43.593540,-79.532750
78,Summerhill West / Rathnelly / South Hill / For...,0.02,1,43.694310,-79.416100
48,Leaside,0.02,1,43.700239,-79.351065
21,Del Ray / Mount Dennis / Keelsdale and Silvert...,0.02,1,43.684038,-79.464550
41,India Bazaar / The Beaches West,0.02,1,43.674130,-79.296440
10,Caledonia-Fairbanks,0.02,1,43.688570,-79.454830
89,Westmount,0.02,1,43.684482,-79.444479
75,Steeles West / L'Amoreaux West,0.02,1,43.798622,-79.308834
19,Davisville,0.02,1,43.701750,-79.383520
53,Moore Park / Summerhill East,0.02,1,43.611850,-79.488738


In [111]:
#cluster 3
print(tnt_merged.loc[tnt_merged['Cluster Labels'] == 2].shape)

tnt_merged.loc[tnt_merged['Cluster Labels'] == 2]

(2, 5)


,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
12,Cedarbrae,0.05,2,43.747728,-79.235174
23,Dorset Park / Wexford Heights / Scarborough To...,0.05,2,43.737040,-79.276940


In [112]:
#cluster 4
print(tnt_merged.loc[tnt_merged['Cluster Labels'] == 3].shape)

tnt_merged.loc[tnt_merged['Cluster Labels'] == 3]

(15, 5)


,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
70,Runnymede / The Junction North,0.01,3,43.663468,-79.481199
6,Birch Cliff / Cliffside West,0.01,3,43.694720,-79.264600
97,York Mills West,0.01,3,43.746475,-79.391617
42,Islington Avenue,0.01,3,43.663213,-79.531510
46,Lawrence Manor / Lawrence Heights,0.01,3,43.723570,-79.437110
66,Rosedale,0.01,3,43.677070,-79.388980
40,Humewood-Cedarvale,0.01,3,43.689420,-79.426980
87,Victoria Village,0.01,3,43.731540,-79.314280
25,Dufferin / Dovercourt Village,0.01,3,43.666368,-79.438120
69,Runnymede / Swansea,0.01,3,43.656357,-79.478395


In [113]:
#cluster 5
print(tnt_merged.loc[tnt_merged['Cluster Labels'] == 4].shape)

tnt_merged.loc[tnt_merged['Cluster Labels'] == 4]

(12, 5)


,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
22,Don Mills,0.03,4,43.705680,-79.333858
39,Humberlea / Emery,0.03,4,43.721510,-79.534010
33,Golden Mile / Clairlea / Oakridge,0.03,4,43.727510,-79.282750
22,Don Mills,0.03,4,43.705680,-79.333858
22,Don Mills,0.03,4,43.705680,-79.333858
22,Don Mills,0.03,4,43.705680,-79.333858
83,Thorncliffe Park,0.03,4,43.707320,-79.343530
20,Davisville North,0.03,4,43.700623,-79.385383
81,The Danforth West / Riverdale,0.03,4,43.690666,-79.290799
67,Roselawn,0.03,4,43.701597,-79.444192


In [114]:
#cluster 6
print(tnt_merged.loc[tnt_merged['Cluster Labels'] == 5].shape)

tnt_merged.loc[tnt_merged['Cluster Labels'] == 5]

(6, 5)


,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
71,Scarborough Village,0.04,5,43.738650,-79.216990
17,Cliffside / Cliffcrest / Scarborough Village West,0.04,5,43.738650,-79.216990
16,Clarks Corners / Tam O'Shanter / Sullivan,0.04,5,43.786430,-79.301560
38,Humber Summit,0.04,5,43.760160,-79.558490
43,Kennedy Park / Ionview / East Birchmount Park,0.04,5,43.713559,-79.260338
51,Milliken / Agincourt North / Steeles East / L'...,0.04,5,43.809300,-79.267070


In [115]:
#cluster 7
print(tnt_merged.loc[tnt_merged['Cluster Labels'] == 6].shape)

tnt_merged.loc[tnt_merged['Cluster Labels'] == 6]

(1, 5)


,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
50,Malvern / Rouge,0.07,6,43.81023,-79.22038


In [116]:
#cluster 8
print(tnt_merged.loc[tnt_merged['Cluster Labels'] == 7].shape)

tnt_merged.loc[tnt_merged['Cluster Labels'] == 7]

(1, 5)


,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
4,Bedford Park / Lawrence Manor East,0.06,7,43.757693,-79.235448


In [117]:
#cluster 9
print(tnt_merged.loc[tnt_merged['Cluster Labels'] == 8].shape)

tnt_merged.loc[tnt_merged['Cluster Labels'] == 8]

(1, 5)


,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
34,Guildwood / Morningside / West Hill,0.040816,8,43.766021,-79.185388


# Therefore as concluded from examining the above clusters :cluster 2,7,8,9 has dearth of indian resturant so it is sensible to open a Indian restaurant in these clusters